# Python Cloud Backup by Paul Chao #

### 提醒，避免收費以及耗費頻寬 :
- 可以加入過濾器 (只傳特殊副檔名)
- 上傳檔案時，注意上傳目錄裡面的檔案數與大小，建議作個測試目錄來玩玩即可
- 下載還原時，可以用 input() 指定寫入目錄，避免現行工作目錄被弄亂

## upload.py ##
把指定目錄中所有的檔案全部放入乾淨的 container

In [5]:
import os
from azure.storage.blob import BlockBlobService

# directory tools
typelist = ('.jpg')

def traverse_dir(entry_dir, service):
    for root, dirs, files in os.walk(entry_dir):
        for file in files:
            fname, fextension = os.path.splitext(file)
            if fextension in typelist:
                p=os.path.join(root,file)
                print(p)
                save_file_to_blob(service, p, p)

# Cloud
storage_account= ??your_storage_account??
storage_key= ??your_storage_key??
container_name= "backup"

def create_service():
    return BlockBlobService(account_name=storage_account, account_key=storage_key)

def save_file_to_blob(service, filepath, blobname):
    service.create_blob_from_path(container_name, blobname, filepath)

def list_file_from_blob(service, containername):
    list = service.list_blobs(containername)
    for blob in list:
        print(blob.name)

def read_file_from_blob(service, blobname, savedname):
    service.get_blob_to_path(container_name, blobname, savedname)

# main procedure
upload_path = input("upload path :")
srv = create_service()
srv.create_container(container_name)

print("Uploading ...")
traverse_dir(upload_path, srv)          

upload path :images
Uploading ...
images\bind-n-connect.jpg
images\git_action.jpg
images\key-value.jpg
images\pubsub.jpg
images\pushpull.jpg
images\reqrep.jpg
images\sql-vs-nosql.jpg


## download.py ##

In [30]:
import os
from azure.storage.blob import BlockBlobService

# directory tools
typelist = ('.jpg')

def traverse_dir(entry_dir, service):
    for root, dirs, files in os.walk(entry_dir):
        for file in files:
            fname, fextension = os.path.splitext(file)
            if fextension in typelist:
                p=os.path.join(root,file)
                print(p)
                save_file_to_blob(service, p, p)

# Cloud
storage_account= ??your_storage_account??
storage_key= ??your_storage_key??

def create_service():
    return BlockBlobService(account_name=storage_account, account_key=storage_key)

def save_file_to_blob(service, filepath, blobname):
    service.create_blob_from_path(container_name, blobname, filepath)

def read_file_from_blob(service, blobname, savedname):
    print("doing read for {},{}".format(blobname, savedname))
    service.get_blob_to_path(container_name, blobname, savedname)

### new function ###
def restore_file_from_blob(service, containername, root_path):
    list = service.list_blobs(containername)
    for blob in list:
        print(blob.name)
        if "/" in blob.name:
        # it is a sub-directory structure
            blob_dir, blob_tail = os.path.split(blob.name)
            print("blob_dir = ", blob_dir)
            #print(blob_tail)
            if os.path.exists(root_path + blob_dir):
            # sub-directory exists
                read_file_from_blob(service, blob.name, root_path+ '/' + blob.name)
            else:
                os.makedirs(root_path+"/"+blob_dir,  exist_ok=True)
                read_file_from_blob(service, blob.name, root_path+ '/' + blob.name)
        else:
        # it is not a sub-directory structure
            read_file_from_blob(service, blob.name, root_path+ '/' + blob.name)
        
# main procedure
container_name = input("restore container name :")
download_path = input("save to path :")
srv = create_service()
#srv.create_container(container_name)

print("Restoring ...")
if os.path.exists(download_path) is False:
    os.mkdir(download_path)
restore_file_from_blob(srv, container_name, download_path)

restore container name :backup
save to path :bktest
Restoring ...
images/bind-n-connect.jpg
blob_dir =  images
doing read for images/bind-n-connect.jpg,bktest/images/bind-n-connect.jpg
images/git_action.jpg
blob_dir =  images
doing read for images/git_action.jpg,bktest/images/git_action.jpg
images/key-value.jpg
blob_dir =  images
doing read for images/key-value.jpg,bktest/images/key-value.jpg
images/pubsub.jpg
blob_dir =  images
doing read for images/pubsub.jpg,bktest/images/pubsub.jpg
images/pushpull.jpg
blob_dir =  images
doing read for images/pushpull.jpg,bktest/images/pushpull.jpg
images/reqrep.jpg
blob_dir =  images
doing read for images/reqrep.jpg,bktest/images/reqrep.jpg
images/sql-vs-nosql.jpg
blob_dir =  images
doing read for images/sql-vs-nosql.jpg,bktest/images/sql-vs-nosql.jpg
